# Sylhet Floods 2022
## download_s1_RTCScenes_hyp3.ipynb
This script downloads the Sentinel-1 radiometric-terrain corrected scenes by running the Hybrid Pluggable Processing Pipeline or ”HyP3” system, from the Alaska Satellite Facility (https://hyp3-docs.asf.alaska.edu/using/sdk/) to get radiometric-terrain corrected images.

In [1]:
import ee

In [2]:
ee.Authenticate()

Enter verification code:  4/1AWtgzh7AG1BedUsaup8uMijhwsTZAaBQQzuFIV2JkuWL-OADqmfmfv4L_wI



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import geemap
import importlib
import shutil
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio import warp
import zipfile
import geopandas as gpd
from osgeo import gdal
import json
from py_linq import Enumerable
from shapely.geometry import box
from shapely.geometry import Polygon

In [5]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data/Raster/')

In [6]:
# Set the module path for helpers scripts
module_path = os.path.abspath(os.path.join('C:/Users/alexa/Documents/GitHub/Sylhet2022Floods/scr/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
# Import modules from the helpers folder
import helpers.prepare_chips as prepChips
importlib.reload(prepChips)

<module 'Helpers.prepare_chips' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\SylhetNASAImpact\\Helpers\\prepare_chips.py'>

## PART 0: Load imports

In [8]:
# Load the FeatureCollectios containing the administrative areas
district = ee.FeatureCollection("projects/ee-nip/assets/ShapeFiles/Admin/Districts")

In [9]:
# Load the ImageCollections for the target satellites / sensors 
S1SeriesName = "COPERNICUS/S1_GRD"
S1pixelsize = 10 # pixel resolution
S1npixels = 256 # 256x256 pixels

## PART 1: Define the ROI

In [10]:
# Define ROIs for the different admin areas containing Sylhet -  we will use District
roi = district.filter(ee.Filter.eq('NAME_2', 'Sylhet'))

## PART 2: Define the TOI

In [11]:
# Define the time period of interest - initially do known date with max flooding
startDate = ee.Date('2022-05-01')
endDate = ee.Date('2022-09-01')

## PART 3: Run the Hyp3 pipeline to get radiometric and terrain-corrected (RTC) images
Run the ASF Hyp3 pipeline to process the raw S1 images and download processed images. Products described here: https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/
We follow the same approach that NASA IMPACT used to create the original dataset used in the ML training: https://docs.google.com/document/d/10lnUwBoNZy9KMwI7UgZ5wi5C-OwKFzSerY193TqNS4Y/edit

"Images were processed to a radiometric and terrain-corrected (RTC) image of the radar amplitude, then converted to a grayscale image for visual analysis using the Hybrid Pluggable Processing Pipeline or ”HyP3” system which takes the Sentinel archive and uses standard SAR image processing procedures to get to a consistent method of generating the VV / VH amplitude or power imagery. The resulting images were in units of dB, uint16 data format."

In [13]:
!pip3 install hyp3_sdk

In [14]:
import hyp3_sdk as sdk

### Authentication

In [15]:
# get credentials automatically
# hyp3 = sdk.HyP3()

# or enter your credentials
hyp3 = sdk.HyP3(prompt=True)

NASA Earthdata Login username:  alexsaunders
NASA Earthdata Login password:  ········


### Get the granules

In [16]:
s1series=ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(startDate, endDate).filterBounds(roi_dist)
granules = s1series.aggregate_array("system:index").getInfo()
print(len(granules), granules)

19 ['S1A_IW_GRDH_1SDV_20220501T234740_20220501T234805_043024_05231C_1E38', 'S1A_IW_GRDH_1SDV_20220511T115642_20220511T115707_043163_0527A8_615B', 'S1A_IW_GRDH_1SDV_20220513T234740_20220513T234805_043199_0528C6_845D', 'S1A_IW_GRDH_1SDV_20220523T115643_20220523T115708_043338_052CDF_38FC', 'S1A_IW_GRDH_1SDV_20220525T234741_20220525T234806_043374_052DFA_89F2', 'S1A_IW_GRDH_1SDV_20220604T115644_20220604T115709_043513_053205_F404', 'S1A_IW_GRDH_1SDV_20220606T234742_20220606T234807_043549_05332B_D43B', 'S1A_IW_GRDH_1SDV_20220616T115645_20220616T115710_043688_05373A_DFAB', 'S1A_IW_GRDH_1SDV_20220618T234743_20220618T234808_043724_05385D_E87E', 'S1A_IW_GRDH_1SDV_20220628T115646_20220628T115711_043863_053C7E_7C9A', 'S1A_IW_GRDH_1SDV_20220630T234744_20220630T234809_043899_053D9A_3654', 'S1A_IW_GRDH_1SDV_20220712T234744_20220712T234809_044074_0542D1_C27A', 'S1A_IW_GRDH_1SDV_20220722T115647_20220722T115712_044213_0546E8_2ED8', 'S1A_IW_GRDH_1SDV_20220724T234745_20220724T234810_044249_054807_4DE4', 'S

### Submit Sentinel-1 RTC jobs

In [17]:
# Key parameters to choose - see https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/
# For function help see https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_rtc_job
pixelspacing = 10 # default 30m, can choose 10m
outputScale = 'decibel' # d=decibel, default = power 

In [20]:
# Run batch jobs
rtc_jobs = sdk.Batch()
for g in granules:
    rtc_jobs += hyp3.submit_rtc_job(granule=g, name='rtc', resolution=pixelspacing, scale=outputScale, speckle_filter=True)
print(rtc_jobs)

19 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 19 pending.


In [18]:
# Return and check the job
rtc_jobs = hyp3.find_jobs(name='rtc')
rtc_jobs = hyp3.watch(rtc_jobs)

  0%|          | 0/21 [timeout in 10800 s]

In [19]:
# Get the succeeded jobs
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = rtc_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

Number of succeeded jobs: 21
Number of failed jobs: 0


In [20]:
download_jobs = succeeded_jobs[0:19]

In [21]:
# Download files for successful jobs - is saved in a zip folder with other outputs
outputPath = rootPath/'SylhetNASAImpact/1_RTC'
outputPath.mkdir(exist_ok=True)
file_list = download_jobs.download_files(location=outputPath, create=True)

  0%|          | 0/19 [00:00<?, ?it/s]

S1A_IW_20220829T234747_DVP_RTC10_G_gdufed_2F04.zip:   0%|          | 0/3936427773 [00:00<?, ?it/s]

S1A_IW_20220827T115649_DVP_RTC10_G_gdufed_53B0.zip:   0%|          | 0/3963374643 [00:00<?, ?it/s]

S1A_IW_20220815T115649_DVP_RTC10_G_gdufed_B11E.zip:   0%|          | 0/3963618939 [00:00<?, ?it/s]

S1A_IW_20220805T234746_DVP_RTC10_G_gdufed_9449.zip:   0%|          | 0/3940331329 [00:00<?, ?it/s]

...

S1A_IW_20220525T234741_DVP_RTC10_G_gdufed_7967.zip:   0%|          | 0/3940100189 [00:00<?, ?it/s]

S1A_IW_20220523T115643_DVP_RTC10_G_gdufed_6B28.zip:   0%|          | 0/3968220813 [00:00<?, ?it/s]

S1A_IW_20220513T234740_DVP_RTC10_G_gdufed_1AB9.zip:   0%|          | 0/3945394952 [00:00<?, ?it/s]

S1A_IW_20220511T115642_DVP_RTC10_G_gdufed_1046.zip:   0%|          | 0/3963149267 [00:00<?, ?it/s]

S1A_IW_20220501T234740_DVP_RTC10_G_gdufed_FDD7.zip:   0%|          | 0/3928051144 [00:00<?, ?it/s]